In [39]:
import pandas as pd
from collections import defaultdict, Counter
from itertools import combinations
import glob
import numpy as np

In [94]:
files = glob.glob('tables/*.tsv')

lst_nodes = []
lst_dfs = []
for filename in files:
    df = pd.read_csv(filename,sep='\t')
    df['lab'] = df['span_normalized'] + " |"+df["label"]+"|"
    for i in df['span_normalized']:
        lst_nodes.append(i)

    fn_dict = defaultdict(list)
    for k,v in pd.Series(df.lab.values,index=df.filename).to_dict().items():
        for i in k.split(','):
            fn_dict[i].append(v)
    d = defaultdict(list)
    for k,v in fn_dict.items():
        for i in combinations(v, 2):
            d[tuple(sorted(i))].append(k)
    df = pd.DataFrame(d.items(), columns=['gram', 'docs'])
    df['from'], df['to'] = list(zip(*((x[0], x[1]) for x in df['gram'].values)))
    df = df.drop(['gram'],axis=1)
    df['count'] = [len(x) for x in df['docs']]
    df['docs'] = [','.join(x) for x in df['docs']]
    df = df[['from','to','count','docs']]
    lst_dfs.append(df)
edges = pd.concat(lst_dfs)

for k,v in Counter(lst_nodes).items():
    if v == 1:
        edges.loc[edges['from'].str.contains(k), 'from'] = str(k)
edges

,from,to,count,docs
0,bilirrubina,creatinina |FARMACO|,1,32611659_ES
1,A,milrinona |FARMACO|,1,32639461_ES
2,A,creatinina |FARMACO|,1,32639064_ES
3,A,ácido acetilsalicílico |FARMACO|,1,32648092B_ES
4,A,dexametasona |FARMACO|,2,"32615807B_ES, 32119083_ES"
5,A,meropenem |FARMACO|,1,32611659_ES
6,A,ácido láctico |FARMACO|,1,32425356_ES
7,A,oxígeno |FARMACO|,1,32637327A_ES
8,A,lactato |FARMACO|,2,"32637327A_ES, 32612435_ES"
9,A,midazolam |FARMACO|,1,32605766_ES


In [72]:
# creates dataframe with co-mentions and dictionary of labels
def comention_tab(filename):
    df = pd.read_csv(filename,sep='\t')
    df["lab"] = df["span_normalized"] + " ("+df["label"].str.lower()+")"
    
    labs = pd.Series(df.label.values,index=df.lab).to_dict()
    labs_multiple = pd.Series(df.label.values,index=df.span_normalized).to_dict()
    
    tot_docs = []
    d = defaultdict(list)
    fn_dict = defaultdict(list)

    for k,v in pd.Series(df.lab.values,index=df.filename).to_dict().items():
        for i in k.split(','):
            fn_dict[i].append(v)
            tot_docs.append(i)
    tot_docs = len(set(tot_docs))

    for k,v in fn_dict.items():
        for i in combinations(v, 2):
            d[tuple(sorted(i))].append(k)
    df = pd.DataFrame(d.items(), columns=['gram', 'docs'])

    df['from'], df['to'] = list(zip(*((x[0], x[1]) for x in df['gram'].values)))
    df = df.drop(['gram'],axis=1)
    df['count'] = [len(x) for x in df['docs']]
    df['docs'] = [','.join(x) for x in df['docs']]
    df = df[['from','to','count','docs']]
    return(df, labs, labs_multiple)

files = glob.glob('tables/*.tsv')
nodes_dict = defaultdict(list)
nodes_dict_multiple = defaultdict(list)
lst = []
for i in files:
    c = comention_tab(i)
    lst.append(c[0])
    for k,v in c[1].items():
        nodes_dict[k].append(v)
    for k,v in c[2].items():
        nodes_dict_multiple[k].append(v)
edges = pd.concat(lst)

nodes = pd.DataFrame(nodes_dict.items(), columns=['id', 'group'])
nodes['title'] = nodes['id']
nodes = nodes[['id','title','group']]
nodes = nodes[(nodes['id'].isin(edges['from']))|(nodes['id'].isin(edges['to']))]

for k,v in nodes_dict_multiple.items():
    if len(v)==1:
        print(edges[edges['from'].str.contains(k)])

                          from                    to  count         docs
1141  alcoholismo (enfermedad)  fumador (enfermedad)      1  32073161_ES
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                      from                          to  count  \
630  de tormenta de citocinas (enfermedad)  trombocitosis (enfermedad)      1   
265                  citocinas (proteinas)      liberación (proteinas)      1   

            docs  
630  32618794_ES  
265  32247642_ES  
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                             from  \
274   choque séptico (enfermedad)   
339   choque séptico (enfermedad)   
484   choque séptico (enferme

Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                            from  \
65               IgG (proteinas)   
74               IgG (proteinas)   
97               IgG (proteinas)   
103              IgG (proteinas)   
120              IgG (proteinas)   
125              IgG (proteinas)   
155  IgG de sarscov2 (proteinas)   
156              IgG (proteinas)   
163              IgG (proteinas)   
171              IgG (proteinas)   
177              IgG (proteinas)   
178              IgG (proteinas)   
180              IgG (proteinas)   
205              IgG (proteinas)   
206              IgG (proteinas)   
213              IgG (proteinas)   
308  IgG de sarscov2 (proteinas)   
341              IgG (proteinas)   
347              IgG (proteinas)   
371              IgG (proteinas)   
379              IgG (proteinas)   
393              IgG (proteinas)   
399              IgG (proteinas)   
440              IgG (proteinas)   
460              IgG (proteinas)   
473  

Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                          from  \
5     insuficiencia renal crónica (enfermedad)   
18            insuficiencia renal (enfermedad)   
206     insuficiencia renal aguda (enfermedad)   
277     insuficiencia renal aguda (enfermedad)   
461     insuficiencia renal aguda (enfermedad)   
738   insuficiencia renal crónica (enfermedad)   
761     insuficiencia renal aguda (enfermedad)   
873           insuficiencia renal (enfermedad)   
1253    insuficiencia renal aguda (enfermedad)   
1259    insuficiencia renal aguda (enfermedad)   
1262    insuficiencia renal aguda (enfermedad)   
1429          insuficiencia renal (enfermedad)   
1495  insuficiencia renal crónica (enfermedad)   

                                                     to  count  \
5                            paro cardíaco (enfermedad) 

                       from                                  to  count  \
66    escalofríos (sintoma)                 orientado (sintoma)      1   
80    escalofríos (sintoma)                 taquipnea (sintoma)      3   
148   escalofríos (sintoma)               hipertermia (sintoma)      1   
196   escalofríos (sintoma)                odinofagia (sintoma)      1   
200   escalofríos (sintoma)                  exantema (sintoma)      1   
252   escalofríos (sintoma)          falta de apetito (sintoma)      1   
260   escalofríos (sintoma)            leucocitopenia (sintoma)      2   
270   escalofríos (sintoma)               somnoliento (sintoma)      1   
307   escalofríos (sintoma)                   vómitos (sintoma)      2   
312   escalofríos (sintoma)              leucocitosis (sintoma)      1   
387   escalofríos (sintoma)                   fumador (sintoma)      1   
413   escalofríos (sintoma)          malestar general (sintoma)      1   
419   escalofríos (sintoma)           

                                          from  \
7           exploración física (procedimiento)   
22        exploración torácica (procedimiento)   
33                 exploración (procedimiento)   
55        exploración cardiaca (procedimiento)   
83          exploración física (procedimiento)   
112         exploración física (procedimiento)   
116         exploración física (procedimiento)   
196       exploración torácica (procedimiento)   
213       exploración cardiaca (procedimiento)   
232         exploración física (procedimiento)   
242         exploración física (procedimiento)   
248         exploración física (procedimiento)   
259         exploración física (procedimiento)   
278        exploración clínica (procedimiento)   
286         exploración física (procedimiento)   
325    exploración neurológica (procedimiento)   
334        exploración clínica (procedimiento)   
389       exploración cardiaca (procedimiento)   
399         exploración física (procedimiento)   


...

                                                   from  \
480   acroisquemia persistente bilateral de las extr...   
555   acroisquemia persistente bilateral de las extr...   
782   acroisquemia persistente bilateral de las extr...   
1018  acroisquemia persistente bilateral de las extr...   
1121  acroisquemia persistente bilateral de las extr...   

                                                to  count         docs  
480   aumento de la cifra de trombocitos (sintoma)      1  32589931_ES  
555                              astenia (sintoma)      1  32589931_ES  
782         síntomas respiratorios leves (sintoma)      1  32589931_ES  
1018                            anasarca (sintoma)      1  32589931_ES  
1121                         proteinuria (sintoma)      1  32589931_ES  
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                    from  \
2     cultivos de esputo (procedimiento)   
1

                                       from  \
102   pruebas hematológicas (procedimiento)   
659   pruebas hematológicas (procedimiento)   
1587  pruebas hematológicas (procedimiento)   

                                         to  count           docs  
102                   rtpcr (procedimiento)      1   32637086B_ES  
659   transfusión de plasma (procedimiento)      1   32637086B_ES  
1587   radiografía torácica (procedimiento)      1   32637086B_ES  
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                      from  \
55    exploración cardiaca (procedimiento)   
213   exploración cardiaca (procedimiento)   
389   exploración cardiaca (procedimiento)   
569   exploración cardiaca (procedimiento)   
669   exploración cardiaca (procedimiento)   
1227  exploración cardiaca (procedimiento)   
1486  exploración cardiaca (procedimiento)   
1530  exploración cardiaca (procedimiento)   
1619  exploración cardiaca (procedimiento)   
1662  exploración ca

Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                    from                                to  count  \
35    disfagia (sintoma)                     tos (sintoma)      2   
73    disfagia (sintoma)                 vómitos (sintoma)      1   
187   disfagia (sintoma)                  febril (sintoma)      1   
431   disfagia (sintoma)        signo de lasègue (sintoma)      1   
580   disfagia (sintoma)           expectoración (sintoma)      1   
671   disfagia (sintoma)                 mialgia (sintoma)      2   
676   disfagia (sintoma)    incapacidad de andar (sintoma)      1   
690   disfagia (sintoma)  síntomas respiratorios (sintoma)      1   
774   disfagia (sintoma)                 náuseas (sintoma)      1   
777   disfagia (sintoma)               hipotonía (sintoma)      1   
904   disfagia (sintoma)    estertores bibasales (sintoma)      1   
917   disfagia (sintoma)                tos seca (sintoma)      2   
920   disfagia (sintoma)                 rig

Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                  from  \
236  anestesia general (procedimiento)   

                                            to  count         docs  
236  profilaxis antitrombótica (procedimiento)      1  32589931_ES  
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
Empty DataFrame
Columns: [from, to, count, docs]
Index: []
                                    from  \
11    fibrilación auricular (enfermedad)   
25    fibrilación auricular (enfermedad)   
41    fibrilación auricular (enfermedad)   
143   fibrilación auricular (enfermedad)   
188   fibrilación auricular (enfermedad)   
246   fibrilación auricular (enfermedad)   
309   fibrilación auricular (enfermedad)   
440   fibrilación auricular (enfermedad)   
452   fibrilación auricular (enfermedad)   
573   fibrilación auricular (enfermedad)  

                            from                           to  count  \
1016  telemetría (procedimiento)  trombólisis (procedimiento)      1   

               docs  
1016  32360439-D_ES  
                  from                                      to  count  \
22    fiebre (sintoma)                   somnolencia (sintoma)      1   
23    fiebre (sintoma)                      letargia (sintoma)      4   
36    fiebre (sintoma)                   somnoliento (sintoma)      2   
43    fiebre (sintoma)                        roncus (sintoma)      1   
94    fiebre (sintoma)                   neutrofilia (sintoma)      1   
126   fiebre (sintoma)                    tabaquismo (sintoma)      1   
164   fiebre (sintoma)            sudoración intensa (sintoma)      1   
221   fiebre (sintoma)                   hipertermia (sintoma)      6   
331   fiebre (sintoma)                     orientado (sintoma)      3   
345   fiebre (sintoma)          incapacidad de andar (sintoma)      1   
372   fi